In [12]:
!pip install transformers
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [14]:
import torch.nn as nn
from transformers import ElectraModel, AutoTokenizer
from transformers import ElectraConfig
import torch

import kss

import pandas as pd

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from tqdm import tqdm
import time

def kss_sentence(sent):
    x = ''
    split_sent = kss.split_sentences(sent)
    for i,s in enumerate(split_sent):
        if i == 0:
            x = s
        else:
            x += ' [SEP] ' + s
    return x

def one_hot_encode(data, n_label=44):
    data = list(map(int,data.split(',')))
    one_hot = [0] * n_label
    label_idx = data
    for idx in label_idx:
        one_hot[idx] = 1
    return torch.LongTensor(one_hot)

def data_preprocessing(df,electra_model):
    tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base", do_lower_case=False)
    X = df['줄거리'][:]
    pr_x = [] # sep 추가
    pr_y = [] # threshold 적용 (one hot)
    pred_y =[] # threshold 미적용
    for x in tqdm(X):
        sp_x = kss_sentence(x)
        embeddings = tokenizer(sp_x, truncation=True, max_length=512, padding="max_length", return_token_type_ids=False, return_attention_mask=True, add_special_tokens=True)

        input_id = torch.LongTensor(embeddings['input_ids']).unsqueeze(0).to('cuda')
        mask = torch.LongTensor(embeddings['attention_mask']).unsqueeze(0).to('cuda')
        sep_idx = torch.where(input_id == 3)

        pr_x.append(sp_x)
        pred,_,_ = electra_model(input_id, mask, sep_idx)
        sig_pred = torch.sigmoid(pred[0])
        
        
        temp = [] # 각 threshold 적용
        for y in sig_pred.tolist():
            if y >= 0.3:
                temp.append(1)
            else:
                temp.append(0)

        pr_y.append(temp)
        pred_y.append(sig_pred.tolist())

    return pr_x, pr_y, pred_y

device = "cuda" if torch.cuda.is_available() else "cpu"

class ELECTRALSTMClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = 'cuda'
        self.config = ElectraConfig.from_pretrained("beomi/KcELECTRA-base",
                                                    problem_type="multi_label_classification",
                                                    num_labels = 44) 
        
        self.embedding_size = 768
        self.batch_size = 32

        self.electra = ElectraModel.from_pretrained("beomi/KcELECTRA-base",config=self.config).to(self.device)
        self.lstm = nn.LSTM(self.embedding_size, self.embedding_size, batch_first=True, bidirectional=True).to(self.device)
        self.fc1 = nn.Linear(self.embedding_size * 5, 44)
        self.fc2 = nn.Linear(self.embedding_size * 2, 44)
        self.gelu = nn.GELU()


    def forward(self, input_ids=None, attention_mask=None, sep_idx=None):
        
        electra_output = self.electra(input_ids, attention_mask)[0]

        cls = electra_output[:, 0, :] # <CLS> embeddings
        # sep 토큰 가져오기
        sep_idx_x = sep_idx[0]
        sep_idx_y = sep_idx[1]

        idx = 0
        cnt = 0
        longest = torch.where(sep_idx_x==torch.mode(sep_idx_x).values)[0].size()[0]
        # 초기화
        sep_embeddings = torch.zeros(cls.size(0), longest, self.embedding_size).to(self.device)

        # embedding 값 집어넣어주기
        for x, y in zip(sep_idx_x, sep_idx_y):
            if idx == x:
                sep_embeddings[x, cnt, :] += electra_output[x, y, :]
                cnt += 1
            else:
                idx += 1
                cnt = 0
                sep_embeddings[x, cnt, :] += electra_output[x, y, :]


        # lstm 실행
        lstm_output, (h, c) = self.lstm(sep_embeddings) # (batch_size, seq_length, embedding_size)

        # lstm 처음과 끝 가져오기
        sep_first = lstm_output[:, 0, :]
        sep_last = lstm_output[:, -1, :]

        # lstm 결과와 cls 토큰 합치기
        concats = torch.cat((cls, sep_first, sep_last), dim=1)
        # fc 레이어에 넣고 44개 output
        x = self.gelu(concats)
        output = self.fc1(x)

        first_output = self.fc2(sep_first)
        last_output = self.fc2(sep_last)

        
        return output, first_output, last_output

electra_model_path= "/content/drive/MyDrive/final_project/data_processing/best_model.pth"

data_type = '영화2'
data_path = '/content/drive/MyDrive/final_project/영화데이터.xlsx'

df = pd.read_excel(data_path)
df_movie = df[['제명', '줄거리']]

trained_model = ELECTRALSTMClassification()
trained_model.load_state_dict(torch.load(electra_model_path)['model_state_dict'],strict=False)
trained_model.to('cuda')


x_data,y_data, y_ori = data_preprocessing(df_movie,trained_model)
data_xy = pd.DataFrame(list(zip(x_data,y_data,y_ori)), columns = ['sep_text','emotion','pb_emotion'])
data_set = pd.concat([df_movie, data_xy], axis=1)

data_set.to_pickle(f"/content/drive/MyDrive/final_project/{data_type}_data.pkl")



Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 53650/53650 [46:28<00:00, 19.24it/s]


In [16]:
data_set

,제명,줄거리,sep_text,emotion,pb_emotion
0,여호,헤이젤은 모든 살아있는 생물을 사랑하는 순진한 시골소녀이다 특히 애완동물 여우를 좋...,헤이젤은 모든 살아있는 생물을 사랑하는 순진한 시골소녀이다 특히 애완동물 여우를 좋...,"[0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, ...","[0.01821475848555565, 0.7304676175117493, 0.79..."
1,이창,사진작가 제프리는 카레이싱 촬영 도중 다리에 부상을 당해 깁스를 하고 있다 밖에 나...,사진작가 제프리는 카레이싱 촬영 도중 다리에 부상을 당해 깁스를 하고 있다 밖에 나...,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, ...","[0.5814776420593262, 0.022592591121792793, 0.0..."
2,대뉴욕과 대로마,로마의 명승지와 미국 뉴욕 도시의 장관을 기록한 문화영화,로마의 명승지와 미국 뉴욕 도시의 장관을 기록한 문화영화,"[0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, ...","[0.023907935246825218, 0.7286266088485718, 0.7..."
3,용서받지 못할자,외딴 곳에서 세 아들 수양딸 레이철과 함께 살고 있는 마틸다 재커리의 집을 찾아와 ...,외딴 곳에서 세 아들 수양딸 레이철과 함께 살고 있는 마틸다 재커리의 집을 찾아와 ...,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0.057931363582611084, 0.3926585018634796, 0.2..."
4,와일드 오키드,젊고 아름다운 법학도 에밀리는 부동산개발회사에 취직하자마자 여사장 클라우디아와 함께...,젊고 아름다운 법학도 에밀리는 부동산개발회사에 취직하자마자 여사장 클라우디아와 함께...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, ...","[0.08139482885599136, 0.22627583146095276, 0.1..."
...,...,...,...,...,...
53645,수조,아버지와 단 둘이 사는 홍연 아버지는 술을 먹으면 홍연을 구타하고 그녀를 돕는 이는...,아버지와 단 둘이 사는 홍연 아버지는 술을 먹으면 홍연을 구타하고 그녀를 돕는 이는...,"[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, ...","[0.4164752662181854, 0.029033221304416656, 0.0..."
53646,민규씨이야기,홀로 고등학생 딸을 키우는 민규씨는 조문객 대행 알바를 하며 근근히 먹고 살고 있다...,홀로 고등학생 딸을 키우는 민규씨는 조문객 대행 알바를 하며 근근히 먹고 살고 있다...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, ...","[0.168670654296875, 0.19945238530635834, 0.133..."
53647,내가 이렇게 외면하고,아픈 노모를 모시고 사는 기초생활수급자 중호는 부정수급으로 인해 지원이 중단된다 이...,아픈 노모를 모시고 사는 기초생활수급자 중호는 부정수급으로 인해 지원이 중단된다 [...,"[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, ...","[0.5481599569320679, 0.030077124014496803, 0.0..."
53648,트리플탱고,파라다이스 빌라 203호에서 오후 2시 부터 2시 30분까지 일어나는 통렬한 블랙 ...,파라다이스 빌라 203호에서 오후 2시 부터 2시 30분까지 일어나는 통렬한 블랙 ...,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, ...","[0.41002556681632996, 0.14545173943042755, 0.0..."
